In [1]:
# Projeto

In [3]:
# importar biblioteca
from pyspark.sql import SparkSession

In [4]:
# vamos criar um contexto spark
contexto_spark = SparkSession.builder.appName("projeto_spark").getOrCreate()

In [5]:
contexto_spark

In [12]:
navios = contexto_spark.read.csv("cruise_ship_info.csv", header=True, inferSchema=True)

In [13]:
navios.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [15]:
navios.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [16]:
# Nosso dataset tem dados categóricos. Como iremos tratar no spark ?

In [21]:
navios.groupby('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [23]:
# Vamos utilizar a coluna Cruise_line, mas ela é um dado categórico. Para resolver isso podemos utilizar StringIndexer
from pyspark.ml.feature import StringIndexer

In [24]:
stringIndexerCruiseLine = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_categorico')

In [28]:
categorico = stringIndexerCruiseLine.fit(navios).transform(navios)

In [30]:
categorico.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_categorico=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_categorico=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_line_categorico=1.0),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, Cruise_line_categorico=1.0),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, Cruise_line_categorico=1.0)]

In [31]:
# Transformar a entrada em Vectors e para isso irei utilizar o VectorAssembler

In [39]:
# importar classe VectorAssembler
from pyspark.ml.feature import VectorAssembler


In [41]:
categorico.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_categorico|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|                  16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|                  16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                   1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|                   1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|                   1.0|
|    Ecstasy|   Carnival| 22|           

In [44]:
categorico.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_categorico']

In [46]:
entrada = [ 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_categorico']

In [47]:
entrada

['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_categorico']

In [65]:
# montamos a estrutura do VectorAssembler
vectorAssembler = VectorAssembler(inputCols=entrada, outputCol="entradas")

In [66]:
# depois você aplica uma transformação utilizando um dataset
# vetor_entradas é um Vector com as entradas 
# target é o alvo
vetor_entradas_target = vectorAssembler.transform(categorico)

In [71]:
# dados_final já contém o vetor  com os dados categóricos
dados_final = vetor_entradas_target.select("entradas", "crew")

# Antes vamos dividir entre treino e teste

In [113]:
# 70%(0.7) para dados de treino e 30%(0.3) para dados de teste
dados_treino, dados_teste = dados_final.randomSplit([0.7, 0.3])

In [114]:
dados_treino.show()

+--------------------+-----+
|            entradas| crew|
+--------------------+-----+
|[5.0,86.0,21.04,9...|  8.0|
|[5.0,115.0,35.74,...| 12.2|
|[5.0,160.0,36.34,...| 13.6|
|[6.0,30.276999999...| 3.55|
|[6.0,30.276999999...| 3.55|
|[6.0,90.0,20.0,9....|  9.0|
|[6.0,93.0,23.94,9...|11.09|
|[6.0,112.0,38.0,9...| 10.9|
|[6.0,113.0,37.82,...| 12.0|
|[7.0,89.6,25.5,9....| 9.87|
|[7.0,158.0,43.7,1...| 13.6|
|[8.0,77.499,19.5,...|  9.0|
|[8.0,110.0,29.74,...| 11.6|
|[9.0,59.058,17.0,...|  7.4|
|[9.0,88.5,21.24,9...| 10.3|
|[9.0,105.0,27.2,8...|10.68|
|[9.0,113.0,26.74,...|12.38|
|[9.0,113.0,26.74,...|12.38|
|[9.0,116.0,26.0,9...| 11.0|
|[10.0,46.0,7.0,6....| 4.47|
+--------------------+-----+
only showing top 20 rows



# Vamos começar aplicar  o modelo de regressão linear no spark

In [115]:
# importar a biblioteca para trabalhar com regressão linear do spark
from pyspark.ml.regression import LinearRegression

In [116]:
# criar uma instância de LinearRegression
# featuresCol foi adicionado porque decidimos nomear o vetor de features como entradas
modelo_navios = LinearRegression(featuresCol='entradas', labelCol='crew')

In [119]:
modelo_navios_treinado = modelo_navios.fit(dados_treino)

In [126]:
# Esta linha poderá mostrar valores zerados caso os coeficientes sejam muito pequenos
modelo_navios_treinado.coefficients

DenseVector([0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 1.0, 0.0])

In [124]:
print(modelo_navios_treinado.coefficients)

[5.775410051734508e-18,4.5194878434003445e-18,-2.989809613366863e-16,4.265452924361341e-16,1.5567376145815986e-15,1.6840862177527558e-17,0.9999999999999986,6.673373013944564e-17]


In [127]:
print(modelo_navios_treinado.intercept)

-2.1292968877056484e-15


In [128]:
resultado_testes = modelo_navios_treinado.evaluate(dados_teste)

In [129]:
resultado_testes.predictions.show()

+--------------------+-----+------------------+
|            entradas| crew|        prediction|
+--------------------+-----+------------------+
|[4.0,220.0,54.0,1...| 21.0|20.999999999999996|
|[5.0,122.0,28.5,1...|  6.7| 6.699999999999997|
|[5.0,133.5,39.59,...|13.13|13.129999999999999|
|[6.0,110.23899999...| 11.5|              11.5|
|[6.0,158.0,43.7,1...| 13.6|              13.6|
|[7.0,116.0,31.0,9...| 12.0|              12.0|
|[8.0,91.0,22.44,9...| 11.0|10.999999999999998|
|[9.0,81.0,21.44,9...| 10.0|              10.0|
|[9.0,85.0,19.68,9...| 8.69|              8.69|
|[9.0,90.09,25.01,...| 8.69|              8.69|
|[9.0,110.0,29.74,...| 11.6|11.600000000000001|
|[10.0,58.825,15.6...|  7.0| 7.000000000000001|
|[10.0,68.0,10.8,7...| 6.36|              6.36|
|[10.0,77.0,20.16,...|  9.0|               9.0|
|[10.0,86.0,21.14,...|  9.2| 9.200000000000001|
|[10.0,105.0,27.2,...|10.68|             10.68|
|[11.0,90.0,22.4,9...| 11.0|10.999999999999998|
|[11.0,91.0,20.32,...| 9.99| 9.989999999

In [130]:
dados_teste.show()

+--------------------+-----+
|            entradas| crew|
+--------------------+-----+
|[4.0,220.0,54.0,1...| 21.0|
|[5.0,122.0,28.5,1...|  6.7|
|[5.0,133.5,39.59,...|13.13|
|[6.0,110.23899999...| 11.5|
|[6.0,158.0,43.7,1...| 13.6|
|[7.0,116.0,31.0,9...| 12.0|
|[8.0,91.0,22.44,9...| 11.0|
|[9.0,81.0,21.44,9...| 10.0|
|[9.0,85.0,19.68,9...| 8.69|
|[9.0,90.09,25.01,...| 8.69|
|[9.0,110.0,29.74,...| 11.6|
|[10.0,58.825,15.6...|  7.0|
|[10.0,68.0,10.8,7...| 6.36|
|[10.0,77.0,20.16,...|  9.0|
|[10.0,86.0,21.14,...|  9.2|
|[10.0,105.0,27.2,...|10.68|
|[11.0,90.0,22.4,9...| 11.0|
|[11.0,91.0,20.32,...| 9.99|
|[11.0,138.0,31.14...|11.85|
|[12.0,42.0,14.8,7...|  6.8|
+--------------------+-----+
only showing top 20 rows



In [94]:
print("MAE", resultado_testes.meanAbsoluteError)

1.853845874996537e-14

In [131]:
print("RMSE", resultado_testes.rootMeanSquaredError)

RMSE 1.203843608254455e-15


In [132]:
print("MSE", resultado_testes.meanSquaredError)

MSE 1.4492394331351055e-30
